# SEO Analysis

Shows charts of how organic search rankings have changed for a given URL

Source: 
- Sistrix ranking data

Overview:
- Clean data
- Plot data

# Step 1: Prepare data

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set the maximum number of characters displayed for each column
pd.set_option('display.max_colwidth', 100)
# Set the maximum number of lines displayed
#pd.set_option('display.max_rows', 500)

# read csv
df = pd.read_csv("./seo-data.csv", sep=";", encoding="utf-8")

df.rename(columns={'ΔClicks': 'ClicksDelta', 'Search Volume': 'SearchVolume'}, inplace=True)

df['ClicksDelta'] = df['ClicksDelta'].astype(float)
df['Position#1'] = df['Position#1'].astype(float)
df['Position#2'] = df['Position#2'].astype(float)
df['SearchVolume'] = df['SearchVolume'].astype(float)
df['PositionDelta'] = df['Position#2'] - df['Position#1']
df['PositionDelta'] = df['PositionDelta'].astype(float)

# create a new column "Type" with the value "Lost" if Position#2 is NaN, "Lower" if Position#2 is higher than Position#1, "Higher" if Position#2 is lower than Position#1
df['Type'] = 'Lower'
df.loc[df['Position#2'].isnull(), 'Type'] = 'Lost'

# set Change to difference between Position#1 and Position#2
df['Change'] = df['Position#1'] - df['Position#2']
df['Change'] = df['Change'].astype(float)
# set Change to 100 if Change is NaN
df.loc[df['Change'].isnull(), 'Change'] = -100
#df.loc[df['Change'] == 'MISS', 'Change'] = -100

# remove entries where SearchVolume is 0
df = df[df.SearchVolume != 0]

# create df_result. This contains the grouped data by URL, the sum of ClicksDelta.
df_result = df.groupby(['URL']).agg({'ClicksDelta': 'sum', 'Change': 'sum', 'SearchVolume': 'sum', 'PositionDelta': 'mean'}).reset_index()
df_result = df_result.sort_values(by=['Change'], ascending=True)

df_result
#df



,URL,ClicksDelta,Change,SearchVolume,PositionDelta
3,https://technik-fuer-kids.de/fitnessuhr-kinder/,-339.0,-7894.0,38690.0,7.615385
12,https://technik-fuer-kids.de/minecraft/,-11.0,-5402.0,26590.0,9.571429
120,https://technik-fuer-kids.de/spielekonsolen-kinder/nintendo-switch-spiele-kinder/,-5.0,-4132.0,4250.0,5.280000
119,https://technik-fuer-kids.de/spielekonsolen-kinder/,-39.0,-3124.0,5220.0,7.294118
125,https://technik-fuer-kids.de/telefonuhr-smartwatch-kinder/,-9.0,-2944.0,45720.0,10.166667
...,...,...,...,...,...
64,https://technik-fuer-kids.de/produkte/need-for-speed-hot-pursuit-remastered-nintendo-switch/,0.0,-4.0,10.0,4.000000
47,https://technik-fuer-kids.de/produkte/lego-43109-vidiyo-metal-dragon-beatbox/,0.0,-4.0,60.0,4.000000
91,https://technik-fuer-kids.de/produkte/samsung-galaxy-a32-5g/,0.0,-3.0,70.0,1.500000
102,https://technik-fuer-kids.de/produkte/trolljaeger-verteidiger-von-arcadia-nintendo-switch/,0.0,-2.0,40.0,2.000000


# Step 2: Plot charts

In [13]:
# create a dropdown menu with the list of unique URLs

# Define the event handler function for the dropdown menu
def dropdown_eventhandler(change):
    with output:
        # clear the output area
        clear_output(wait=True)
        
        # filter the dataframe based on the selected URL
        filtered_df = df[df['URL'] == change.new]
        
        # create subplots
        fig, axs = plt.subplots(1, 2, figsize=(10, 4))
        
        # plot histogram for 'Change' column
        axs[0].hist(filtered_df['Change'], bins=20)
        axs[0].set_xlabel('Change')
        axs[0].set_ylabel('Count of Keywords')
        axs[0].set_title('Change Histogram')
        
        # plot histogram for 'Position#1' and 'Position#2' columns
        axs[1].hist([filtered_df['Position#1'], filtered_df['Position#2']], bins=10, label=['Position#1', 'Position#2'])
        axs[1].legend(loc='upper right')
        axs[1].set_xlabel('Position')
        axs[1].set_ylabel('Count of Keywords')
        axs[1].set_title('Position Histogram')
        
        # adjust spacing between subplots
        plt.tight_layout()
        
        # display the plot
        plt.show()
        
        # display the table of keywords and metrics
        display(filtered_df)
        
        # create a list of unique URLs where Change > 500
        urls = df[df['Change'] > 500]['URL'].unique().tolist()

        # create a dropdown menu with the list of unique URLs
        dropdown = widgets.Dropdown(options=urls, description='URL:')

from ipywidgets import widgets
from IPython.display import display, clear_output

# create a list of unique URLs
urls = df.URL.unique().tolist()

dropdown = widgets.Dropdown(options=urls, description='URL:')
dropdown.layout.width = '500px'

output = widgets.Output()
display(dropdown, output)

dropdown.observe(dropdown_eventhandler, names='value')


Dropdown(description='URL:', layout=Layout(width='500px'), options=('https://technik-fuer-kids.de/tablets-kind…

Output()